<a href="https://colab.research.google.com/github/anslemhuncho/Crowd-Epidemiology-/blob/master/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.9 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Define paths to your train and test folders
train_path = '/content/drive/MyDrive/Normalised data/train'
test_path = '/content/drive/MyDrive/Normalised data/test'

# Set up data generators with data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1  # Add validation split here
)

# Adjust batch size based on your system's memory
batch_size = 32
# Define the list of class labels corresponding to your dataset
class_labels = ['animals_protection', 'appealing_for_peace', 'brexit', 'covid19', 'election', 'environment_protection', 'gender_equality', 'immigration', 'racial_equality', 'yellow_vests_movement']

# Generate batches of augmented data from train and test folders with validation split
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(254, 254),
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_labels,  # Manually specify class labels
    shuffle=True,
    subset='training'  # Specify subset as training for train data
)

validation_generator = datagen.flow_from_directory(
    train_path,
    target_size=(254, 254),
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_labels,  # Manually specify class labels
    shuffle=False,
    subset='validation'  # Specify subset as validation for validation data
)

test_generator = datagen.flow_from_directory(
    test_path,
    target_size=(254, 254),
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_labels,  # Manually specify class labels
    shuffle=False
)

# Define the ResNet model architecture
def build_resnet_model(hp):
    base_model = ResNet50(include_top=False, weights=None, input_shape=(254, 254, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(hp.Int('dense_units', min_value=64, max_value=256, step=32), activation='relu')(x)
    x = Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1))(x)
    predictions = Dense(len(class_labels), activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Set up the tuner for ResNet model
resnet_tuner = RandomSearch(
    build_resnet_model,
    objective='accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='resnet_hyperparameter_tuning',
    project_name='crowd_prediction_resnet'
)

# Perform hyperparameter tuning for ResNet model
resnet_tuner.search(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[EarlyStopping(patience=3)]
)

# Get the best ResNet model and its hyperparameters
best_resnet_model = resnet_tuner.get_best_models(num_models=1)[0]
best_resnet_hyperparameters = resnet_tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters for ResNet
print(f'Best Hyperparameters for ResNet: {best_resnet_hyperparameters}')

# Train the best ResNet model using the full training data (including validation)
best_resnet_model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Evaluate the best ResNet model on the test data
test_loss_resnet, test_accuracy_resnet = best_resnet_model.evaluate(test_generator)
print(f'Test Accuracy of Best ResNet Model: {test_accuracy_resnet}')

# Plot training history of the best ResNet model
plt.plot(best_resnet_model.history.history['accuracy'], label='accuracy')
plt.plot(best_resnet_model.history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()


Found 3747 images belonging to 10 classes.
Found 411 images belonging to 10 classes.
Found 2069 images belonging to 10 classes.
Reloading Tuner from resnet_hyperparameter_tuning/crowd_prediction_resnet/tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
128               |128               |dense_units
0.1               |0.1               |dropout
0.0001            |0.0001            |learning_rate

Epoch 1/10
